In [25]:
#Converting TeleQNA.txt to dictionary format
import ast
import json
with open("TeleQnA.txt", "r") as f:
    content = f.read()

nested_dict = ast.literal_eval(content)

data_set = []

for item in nested_dict:
    question_num = item
    question = nested_dict.get(item).get("question")
    answer = nested_dict.get(item).get("answer").replace("option 1", "A.").replace("option 2", "B.").replace("option 3", "C.").replace("option 4", "D.").replace("option 5", "E.")
    option1 = "A. " + nested_dict.get(item).get("option 1")
    option2 = "B. "+nested_dict.get(item).get("option 2")
    option3 = "C. "+nested_dict.get(item).get("option 3", "")
    option4 = "D. "+nested_dict.get(item).get("option 4", "")
    option5 = "E. "+nested_dict.get(item).get("option 5","")
    explanation = nested_dict.get(item).get("explanation")
    # data_set.append({
    #     "question": question + "\n"+ option1 +" "+ option2+ " "+ option3 +" "+ option4 +" "+ option5,
    #     "answer": "The correct answer is "+answer+" Explanation: "+explanation,
    # })

    full_text = f"{question}\n ### Response:\n The correct answer is : "+ answer + ". Because: " +  explanation
    data_set.append({"text": full_text})


# with open("TeleQnA.json", "w") as f:
#     json.dump(result_dict, f,indent=4)

print(data_set[0])


{'text': 'What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]\n ### Response:\n The correct answer is : B.: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints. Because: The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.'}


## Step 0
### import

In [26]:
from datasets import load_dataset, DatasetDict, Dataset

full_data = Dataset.from_list(data_set)



## Step 1
### Import TeleQNA into a list

#### Example 1:
##### [{'question 1': 'What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]',
##### 'answer':'To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints'},
##### 'explaination': The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints. ']

## Step 2:
### Load Model & Tokenizer

In [27]:
from transformers import AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Required for GPT-2

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = full_data.map(tokenize)

Map: 100%|██████████| 10000/10000 [00:04<00:00, 2064.86 examples/s]


## Step 3:
### Tokenize Dataset

In [28]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


d:\lc\tele_llm\fine-tune\lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## Step 3:
### Tokenize Dataset

In [29]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="teleqna-lora-gpt2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=False,
    logging_steps=500,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

# Save your model
model.save_pretrained("teleqna-lora-gpt2")
tokenizer.save_pretrained("teleqna-lora-gpt2")

C:\Users\lc928\AppData\Local\Temp\ipykernel_3952\1008433669.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/7500 [00:00<?, ?it/s]d:\lc\tele_llm\fine-tune\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 10/7500 [00:04<29:50,  4.18it/s] 

{'loss': 4.2422, 'grad_norm': 1.6353472471237183, 'learning_rate': 1.9973333333333334e-05, 'epoch': 0.0}


  0%|          | 20/7500 [00:06<25:40,  4.86it/s]

{'loss': 4.2968, 'grad_norm': 2.0252890586853027, 'learning_rate': 1.9946666666666667e-05, 'epoch': 0.01}


  0%|          | 31/7500 [00:09<25:43,  4.84it/s]

...

100%|██████████| 7500/7500 [26:32<00:00,  4.63it/s]

{'loss': 2.3354, 'grad_norm': 5.323387145996094, 'learning_rate': 0.0, 'epoch': 3.0}


100%|██████████| 7500/7500 [26:34<00:00,  4.70it/s]

{'train_runtime': 1594.1251, 'train_samples_per_second': 18.819, 'train_steps_per_second': 4.705, 'train_loss': 2.500469948577881, 'epoch': 3.0}


('teleqna-lora-gpt2\\tokenizer_config.json',
 'teleqna-lora-gpt2\\special_tokens_map.json',
 'teleqna-lora-gpt2\\vocab.json',
 'teleqna-lora-gpt2\\merges.txt',
 'teleqna-lora-gpt2\\added_tokens.json',
 'teleqna-lora-gpt2\\tokenizer.json')

In [31]:
from transformers import pipeline

pipe = pipeline("text-generation", model="teleqna-lora-gpt2", tokenizer=tokenizer)


test_question = "Question: What is 5g?"
output = pipe(test_question, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.7)

print(output[0]["generated_text"])



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Question: What is 5g?

Answer: 5g is a type of energy that is present in the bloodstream, but it is not stored in the body.

5g is one of the most common and recognizable types of energy. 5g is not a form of energy, but rather it is a natural, natural state of being.

5g is a type of energy that is present in the bloodstream, but it is not stored in the body.

5g is a type of energy that is
